In [1]:
from google.colab import drive 
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Colab Notebooks/4332proj3

/content/drive/MyDrive/Colab Notebooks/4332proj3


In [3]:
!pip install tensorflow==2.1.0

     |████████████████████████████████| 421.8MB 32kB/s 
     |████████████████████████████████| 3.9MB 36.5MB/s 
     |████████████████████████████████| 450kB 34.6MB/s 
     |████████████████████████████████| 51kB 5.5MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7540 sha256=0864d6d844071682ce6fb993ffe272e7e70227b82397ee6394da7a86f4ba84ef
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled tensorboard-2.4.1
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully 

In [4]:
!pip install keras==2.3.1

     |████████████████████████████████| 378kB 5.8MB/s 
  Found existing installation: Keras 2.4.3
    Uninstalling Keras-2.4.3:
      Successfully uninstalled Keras-2.4.3


In [5]:
from collections import Counter
from itertools import combinations
from math import sqrt
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0" #comment out this line if you want to use gpu
import random
from keras.layers import Concatenate, Dense, Dot, Dropout, Embedding, Input, Reshape
from keras.models import Model
from keras.callbacks import Callback, ModelCheckpoint
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import tensorflow
import pandas as pd
import ast
import math

Using TensorFlow backend.


### Set random seed

In [6]:
import random
random.seed(2021)
np.random.seed(2021)
# tf > 2.0
tensorflow.random.set_seed(2021)
#tf < 2.0
#tf.set_random_seed(2021)

### Root Mean Squared Error (RMSE) is used to evaluate the performance of a recommendation algorithm, so we need to define the following utility function to compute the RMSE given the predicted ratings and the ground truth ratings. 

In [7]:
'''
params:
    -pred: an array containing all predicted ratings
    -actual: an array containing all ground truth ratings
    
return:
    a scalar whose value is the rmse
'''
def rmse(pred, actual):
    # Ignore ratings with value zero.
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return sqrt(mean_squared_error(pred, actual))

# Implement Wide and Deep Learning (WDL) Model

### The wide component is a generalized linear model that takes in the raw input features and the cross-product transformation of categorical features, which enables it to learn the frequent co-occurrence of items or features. 

### The deep component is a Feed-forward Neural Network (FNN) which takes in both continuous and categorical features as input. Specifically,  the normalized values of continuous features are concatenated with the low-dimensional dense embedding vectors converted from categorical features. This concatenated vector is then fed into the FNN during each foward pass. This mechanism tend to increase the diversity of recommendations.

In [8]:
'''
params:
    -len_continuous: number of continuous features
    -deep_vocab_lens: an array of integers where deep_vocab_lens[i] represents the number of unique values of (i+1)-th deep 
        categorical feature
    -len_wide: number of wide features
    -embed_size: dimension of the embedding vectors of deep categorical features
    
return:
    a keras Model object for the constructed wdl model 
'''


def build_wdl_model(len_continuous, deep_vocab_lens, len_wide, embed_size):
    # A list containing all input layers
    input_list = []
    
    # Input layer for continuous features
    continuous_input = Input(shape=(len_continuous,), dtype='float32', name='continuous_input')
    input_list.append(continuous_input)
    
    
    # Get embeddings for all deep categorical features
    emb_list = []
    for vocab_size in deep_vocab_lens:
        _input = Input(shape=(1,), dtype='int32')
        input_list.append(_input)
        _emb = Embedding(output_dim=embed_size, input_dim=vocab_size, input_length=1)(_input)
        _emb = Reshape((embed_size,))(_emb)
        emb_list.append(_emb)
    
    
   
    # Create input layer for deep component by concatenating the embeddings and continuous features' input layer
    deep_input = Concatenate()(emb_list + [continuous_input])
    

    # Construct deep component
    dense_1 = Dense(256, activation='relu')(deep_input)
    dense_1_dp = Dropout(0.5)(dense_1)
    dense_2 = Dense(128, activation='relu')(dense_1_dp)
    dense_2_dp = Dropout(0.2)(dense_2)
    dense_3 = Dense(64, activation='relu')(dense_2_dp)
    dense_3_dp = Dropout(0.2)(dense_3)
    # dense_3_dp = dense_3


    
    # Create input layer for wide component
    wide_input = Input(shape=(len_wide,), dtype='float32')
    input_list.append(wide_input)

    
    # Concatenate the outputs of deep and wide components and feed the 
    # concatenated vector into the finall fully connected layer
    fc_input = Concatenate()([dense_3_dp, wide_input])
    model_output = Dense(1)(fc_input)
    
    model = Model(inputs=input_list,
                  outputs=model_output)
    return model
    

# Utility functions to get the values of different types of features

### Continuous features

In [9]:
'''
params:
    -df: input dataframe
    -continuous_columns: column names of continuous features
    
return: 
    a numpy array where each row contains the values of continuous features in the corresponding row of the
    input dataframe
'''
def get_continuous_features(df, continuous_columns):
    continuous_features = df[continuous_columns].values
    return continuous_features

### Cross product transformation of categorical features

In [10]:
'''
params:
    -df: input dataframe
    -comb_p: number of elements in each combination (e.g., there are two elements in the combination {fried chicken, chicken and 
    waffle}, and three elements in the combination {fried chicken, chicken and waffle, chicken fried rice})
    -topk: number of mostly frequent combinations to retrieve
    -output_freq: whether to return the frequencies of retrieved combinations
    
return:
    1. output_freq = True: a list X where each element is a tuple containing a combinantion tuple and corresponding frequency, and the 
        elements are stored in the descending order of their frequencies
    2. output_freq = False: a list X where each element is a tuple containing a combinantion tuple, and the elements are stored in 
    the descending order of their frequencies
'''
def get_top_k_p_combinations(df, comb_p, topk, output_freq=False):
    # get all combinations with comb_p
    def get_category_combinations(categories_str, comb_p=2):
        categories = categories_str.split(', ')
        return list(combinations(categories, comb_p))
    # [('Lounges', 'Dance Clubs'), ('Lounges', 'Bars'), ('Lounges', 'Nightlife'), ('Dance Clubs', 'Bars'), ('Dance Clubs', 'Nightlife'), ('Bars', 'Nightlife')]
    all_categories_p_combos = df["item_categories"].apply(
        lambda x: get_category_combinations(x, comb_p)).values.tolist()
    # ('Lounges', 'Dance Clubs')
    # list of tuples that each index refer to one combination
    all_categories_p_combos = [tuple(t) for item in all_categories_p_combos for t in item]

    tmp = dict(Counter(all_categories_p_combos))
    sorted_categories_combinations = list(sorted(tmp.items(), key=lambda x: x[1], reverse=True))
    if output_freq:
        return sorted_categories_combinations[:topk]
    else:
        return [t[0] for t in sorted_categories_combinations[:topk]]

### Wide features

In [11]:
'''
params:
    -df: input dataframe
    -selected_categories_to_idx: a dictionary mapping item categories to corrresponding integral indices
    -top_combinations: a list containing retrieved mostly frequent combinantions of item categories
    
return:
    a numpy array where each row contains the categorical features' binary encodings and cross product
    transformations for the corresponding row of the input dataframe
'''

def get_wide_features(df, selected_categories_to_idx, top_combinations):
    def categories_to_binary_output(categories):
        binary_output = [0 for _ in range(len(selected_categories_to_idx))]
        for category in categories.split(', '):
            if category in selected_categories_to_idx:
                binary_output[selected_categories_to_idx[category]] = 1
            else:
                binary_output[0] = 1
        return binary_output
    def categories_cross_transformation(categories):
        current_category_set = set(categories.split(', '))
        corss_transform_output = [0 for _ in range(len(top_combinations))]
        for k, comb_k in enumerate(top_combinations):
            if len(current_category_set & comb_k) == len(comb_k):
                corss_transform_output[k] = 1
            else:
                corss_transform_output[k] = 0
        return corss_transform_output

    category_binary_features = np.array(df.item_categories.apply(
        lambda x: categories_to_binary_output(x)).values.tolist())
    print('category_binary_features shape:',category_binary_features.shape)
    category_corss_transform_features = np.array(df.item_categories.apply(
        lambda x: categories_cross_transformation(x)).values.tolist())
    print('category_cross_features shape:',category_corss_transform_features.shape)
    out = np.concatenate((category_binary_features, category_corss_transform_features), axis=1)
    print('wide features shape:',out.shape)
    return np.concatenate((category_binary_features, category_corss_transform_features), axis=1)


# Rating Prediction

### Load train, validation and test rating tables

In [12]:
def isnan(value):
    try:
        return math.isnan(float(value))
    except:
        return False

In [13]:
tr_df = pd.read_csv("data/train.csv")
val_df = pd.read_csv("data/valid.csv")
te_df = pd.read_csv("data/test.csv")

tr_ratings = tr_df.stars.values
val_ratings = val_df.stars.values



### Load content feautures tables of users and items

In [14]:
user_df = pd.read_csv("data/user.csv")
item_df = pd.read_csv("data/business.csv")

# Rename some columns of dfs and convert the indices of dfs into string type for easier reference in later stage 
user_df = user_df.rename(index=str, columns={t: 'user_' + t for t in user_df.columns if t != 'user_id'})
item_df = item_df.rename(index=str, columns={t: 'item_' + t for t in item_df.columns if t != 'business_id'})


In [15]:
item_df

,item_Unnamed: 0,business_id,item_name,item_address,item_city,item_state,item_postal_code,item_latitude,item_longitude,item_stars,item_review_count,item_is_open,item_attributes,item_categories,item_hours
0,0,de9d8b06461fa8193081e69d4e3ae345,Apteka,4606 Penn Ave,Pittsburgh,PA,15224,40.465694,-79.949324,4.5,242,1,"{'CoatCheck': 'False', 'BusinessParking': ""{'g...","Nightlife, Bars, Polish, Modern European, Rest...","{'Wednesday': '17:0-0:0', 'Thursday': '17:0-0:..."
1,1,334a85d83ec4545b5b2b4581e5182c47,Nee House Chinese Restaurant,"13843 N Tatum Blvd, Ste 15",Phoenix,AZ,85032,33.613020,-111.977036,3.5,269,1,"{'Caters': 'True', 'GoodForKids': 'True', 'Noi...","Chinese, Restaurants","{'Monday': '11:0-21:0', 'Tuesday': '11:0-21:0'..."
2,2,d39623cbcc490dff7ec1036a4f43a337,Vintage 95,95 W Boston,Chandler,AZ,85225,33.302093,-111.842613,4.0,320,0,"{'OutdoorSeating': 'True', 'HasTV': 'True', 'N...","American (New), Bars, Wine Bars, Nightlife, Re...","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'..."
3,3,218d525e3a7223a9e2265c2ea481116d,Served,"1770 W Horizon Ridge, Ste 100",Henderson,NV,89012,36.010745,-115.064803,4.5,664,1,"{'DriveThru': 'False', 'RestaurantsAttire': ""'...","Ethnic Food, American (New), Burgers, Food, Re...","{'Monday': '0:0-0:0', 'Tuesday': '9:0-15:0', '..."
4,4,d5062bde99f4003b7b7ef546c16cf0cc,JJ's Red Hots - Dilworth,1514 East Blvd,Charlotte,NC,28203,35.199798,-80.842295,4.0,380,1,"{'RestaurantsReservations': 'True', 'BusinessA...","Caterers, Hot Dogs, Restaurants, Vegetarian, A...","{'Monday': '11:0-21:0', 'Tuesday': '11:0-21:0'..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5959,5959,bfa5e2aab6f787e190082cf76e748203,Naked City Pizza Shop,3240 S Arville St,Las Vegas,NV,89102,36.130804,-115.198971,4.0,576,1,"{'BusinessParking': ""{'garage': False, 'street...","Sandwiches, Pizza, Chicken Wings, Italian, Res...","{'Monday': '12:0-21:0', 'Tuesday': '12:0-21:0'..."
5960,5960,bf46e96044d23cccab27cf4534af2add,Pomo Pizzeria Gilbert,"366 N Gilbert Rd, Ste 106",Gilbert,AZ,85234,33.356542,-111.790019,4.0,337,1,"{'WheelchairAccessible': 'True', 'WiFi': ""u'fr...","Restaurants, Italian, Pizza","{'Monday': '0:0-0:0', 'Tuesday': '11:0-22:0', ..."
5961,5961,921ae754074c121b7aa9523ca7e8b7cd,Greens and Proteins,"8975 S Eastern Ave, Ste 3C",Las Vegas,NV,89123,36.026212,-115.119341,4.0,704,1,"{'Alcohol': ""u'none'"", 'BusinessParking': ""{'g...","Sandwiches, Cafes, Pizza, Vegetarian, Gluten-F...","{'Monday': '9:30-19:30', 'Tuesday': '7:0-22:0'..."
5962,5962,8e350f4ad2441016656c1a5bcf023190,Jessie Rae's BBQ,5611 S Valley View Blvd,Las Vegas,NV,89118,36.087895,-115.190329,4.5,595,1,"{'Caters': 'True', 'RestaurantsTableService': ...","Farmers Market, Caterers, Food, Street Vendors...","{'Monday': '0:0-0:0', 'Tuesday': '11:30-20:30'..."


In [16]:
for i,dict_str in enumerate(item_df["item_attributes"]):
    # check for nan
    if(isnan(dict_str)):
        print(f'line {i+1} has nan index')
#         print('**'*20)
#         print(dict_str)
    else:
        dict_str = dict_str.replace('"','')
        temp_dict = ast.literal_eval(dict_str)  
        item_df["item_attributes"][i] = temp_dict

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


line 44 has nan index
line 177 has nan index
line 199 has nan index
line 335 has nan index
line 385 has nan index
line 401 has nan index
line 404 has nan index
line 488 has nan index
line 550 has nan index
line 624 has nan index
line 720 has nan index
line 770 has nan index
line 1065 has nan index
line 1112 has nan index
line 1130 has nan index
line 1134 has nan index
line 1191 has nan index
line 1384 has nan index
line 1413 has nan index
line 1506 has nan index
line 1510 has nan index
line 1775 has nan index
line 1883 has nan index
line 1885 has nan index
line 2024 has nan index
line 2349 has nan index
line 2375 has nan index
line 2415 has nan index
line 2429 has nan index
line 2476 has nan index
line 2506 has nan index
line 2601 has nan index
line 2802 has nan index
line 2823 has nan index
line 3232 has nan index
line 3269 has nan index
line 3356 has nan index
line 3378 has nan index
line 3549 has nan index
line 3714 has nan index
line 4086 has nan index
line 4155 has nan index
line 

### Associate each row in the rating tables with corresponding user's and item's content features through merging the rating tables and content features tables

In [17]:
# Save the original row indices of each rating table
tr_df["index"] = tr_df.index
val_df["index"]  = val_df.index
te_df["index"] = te_df.index

tr_df = pd.merge(pd.merge(tr_df, user_df, on='user_id'), item_df, on='business_id').sort_values(by=['index']).reset_index(drop=True)
val_df = pd.merge(pd.merge(val_df, user_df, on='user_id'), item_df, on='business_id').sort_values(by=['index']).reset_index(drop=True)
te_df = pd.merge(pd.merge(te_df, user_df, on='user_id'), item_df, on='business_id').sort_values(by=['index']).reset_index(drop=True)

### Prepare continuous features

In [18]:
# Specify the columns containing conitnuous features
continuous_columns = ["user_average_stars", "user_cool", "user_fans", 
                      "user_review_count", "user_useful", "user_funny",
                      "item_is_open", "item_latitude", "item_longitude", 
                      "item_review_count", "item_stars"]

# Get values of continous features for train/validation/test sets using the utility function defined previously

tr_continuous_features = get_continuous_features(tr_df, continuous_columns)
val_continuous_features = get_continuous_features(val_df, continuous_columns)
te_continuous_features = get_continuous_features(te_df, continuous_columns)

# Standardize each feature by removing the mean of the training samples and scaling to unit variance.
# See https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html for more details.
scaler = StandardScaler().fit(tr_continuous_features)

tr_continuous_features = scaler.transform(tr_continuous_features)
val_continuous_features = scaler.transform(val_continuous_features)
te_continuous_features = scaler.transform(te_continuous_features)

### Prepare deep categorical features

In [19]:
# Sepcify column names of deep categorical features
item_deep_columns = ["item_city", "item_postal_code", "item_state"]

# An array of integers where deep_vocab_lens[i] represents the number of unique values of (i+1)-th deep categorical feature
item_deep_vocab_lens = []

for col_name in item_deep_columns:
    # Get all unique values of this deep categorical feature
    tmp = item_df[col_name].unique()
    
    # Create a dictionary mapping each unique value to a unique integral index
    vocab = dict(zip(tmp, range(1, len(tmp) + 1)))
    
    # Get the number of unique values of this deep categorical features
    item_deep_vocab_lens.append(len(vocab) + 1)
    
    # Create a new column where each entry stores the integral index of this deep categorical feature's value in the same row
    item_df[col_name + "_idx"] = item_df[col_name].apply(lambda x: vocab[x])


# Create a dictionary mapping each business id to corresponding values of deep categorical features
item_deep_idx_columns = [t + "_idx" for t in item_deep_columns]
item_to_deep_categorical_features = dict(zip(item_df.business_id.values, item_df[item_deep_idx_columns].values.tolist()))

# Creat numpy arrays storing corresponding deep categorical features' values of train/validation/test sets using the above mapping
tr_deep_categorical_features = np.array(tr_df.business_id.apply(lambda x: item_to_deep_categorical_features[x]).values.tolist())
val_deep_categorical_features = np.array(val_df.business_id.apply(lambda x: item_to_deep_categorical_features[x]).values.tolist())
te_deep_categorical_features = np.array(te_df.business_id.apply(lambda x: item_to_deep_categorical_features[x]).values.tolist())


### Prepare wide features

##### Prepare binary encoding for each selected category

In [20]:
# Collect the categories of all items 
all_categories = [category for category_list in item_df.item_categories.values for category in category_list.split(", ")]

# Sort all unique values of the item categories by their frequencies in descending order
category_sorted = sorted(Counter(all_categories).items(), key=lambda x: x[1], reverse=True)

# Select top 500 most frequent categories
selected_categories = [t[0] for t in category_sorted[:500]]

# Create a dictionary mapping each secleted category to a unique integral index
selected_categories_to_idx = dict(zip(selected_categories, range(1, len(selected_categories) + 1)))

# Map all categories unseen in the item df to index 0
selected_categories_to_idx['unk'] = 0

# Create a dictionary mapping each integral index to corresponding category
idx_to_selected_categories = {val: key for key, val in selected_categories_to_idx.items()}


##### Prepare cross product transformation for categories

In [21]:
# Get most frequent categories combinantions using the utility function defined previously and store them in the folloing list
top_combinations = []

# Get top 50 most frequent two-categories combinantions in the train set

top_combinations += get_top_k_p_combinations(tr_df, 2, 50, output_freq=False)

# Get top 30 most frequent three-categories combinantions in the train set
top_combinations += get_top_k_p_combinations(tr_df, 3, 30, output_freq=False)

# Get top 20 most frequent four-categories combinantions in the train set
top_combinations += get_top_k_p_combinations(tr_df, 4, 20, output_freq=False)

# Convert each combinantion in the list to a set data structure
top_combinations = [set(t) for t in top_combinations]

In [22]:
# Get values of wide features for train/validation/test sets using the utility function defined previously

tr_wide_features = get_wide_features(tr_df, selected_categories_to_idx, top_combinations)
val_wide_features = get_wide_features(val_df, selected_categories_to_idx, top_combinations)
te_wide_features = get_wide_features(te_df, selected_categories_to_idx, top_combinations)


category_binary_features shape: (60080, 501)
category_cross_features shape: (60080, 100)
wide features shape: (60080, 601)
category_binary_features shape: (7510, 501)
category_cross_features shape: (7510, 100)
wide features shape: (7510, 601)
category_binary_features shape: (7510, 501)
category_cross_features shape: (7510, 100)
wide features shape: (7510, 601)


### Build the input list for each of the train/validation/test sets through aggregating all continuous, deep categorical and wide features


In [23]:
# tr_features = [tr_continuous_features,categorical_features_0,categorical_features_1,categorical_features_2,tr_wide_features]
tr_features = []
tr_features.append(tr_continuous_features.tolist())
tr_features += [tr_deep_categorical_features[:,i].tolist() for i in range(tr_deep_categorical_features.shape[1])]
tr_features.append(tr_wide_features.tolist())



val_features = []
val_features.append(val_continuous_features.tolist())
val_features += [val_deep_categorical_features[:,i].tolist() for i in range(val_deep_categorical_features.shape[1])]
val_features.append(val_wide_features.tolist())

te_features = []
te_features.append(te_continuous_features.tolist())
te_features += [te_deep_categorical_features[:,i].tolist() for i in range(te_deep_categorical_features.shape[1])]
te_features.append(te_wide_features.tolist())

In [24]:
len(te_features[0])

7510

### Build the WDL model defined above

In [25]:
wdl_model = build_wdl_model(
        len(tr_continuous_features[0]),
        item_deep_vocab_lens,   # num of category classes
        len(tr_wide_features[0]), 
        embed_size=100)
print(len(tr_continuous_features[0]))
print(item_deep_vocab_lens)
print(len(tr_wide_features[0]))

11
[118, 799, 14]
601


### Train the model using Adagrad optimizer and mean squared error loss

In [26]:
wdl_model.compile(optimizer='SGD', loss='mse')


history = wdl_model.fit(
    tr_features, 
    tr_ratings, 
        epochs=5, verbose=1, callbacks=[ModelCheckpoint('model.h5')])

/usr/local/lib/python3.7/dist-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/5
60080/60080 [==============================] - 9s 145us/step - loss: 1.2027
Epoch 2/5
60080/60080 [==============================] - 8s 141us/step - loss: 1.0574
Epoch 3/5
60080/60080 [==============================] - 8s 140us/step - loss: 1.0419
Epoch 4/5
60080/60080 [==============================] - 8s 138us/step - loss: 1.0322
Epoch 5/5
60080/60080 [==============================] - 9s 142us/step - loss: 1.0287


### Evaluate the model on train and validation sets using RMSE¶

In [27]:
y_pred = wdl_model.predict(tr_features)
print("TRAIN RMSE: ", rmse(y_pred, tr_ratings))
y_pred = wdl_model.predict(val_features)
print("VALID RMSE: ", rmse(y_pred, val_ratings))


TRAIN RMSE:  1.004384287124796
VALID RMSE:  0.9978442084793959


In [28]:
res_df_val = pd.read_csv("data/valid.csv")
res_df_val
res_df_val['stars'] = y_pred[:, 0]
NAME = 'val_pre'
res_df_val.to_csv("{}.csv".format(NAME), index=False)
print("Writing val predictions to file done.")

Writing val predictions to file done.


In [29]:
y_pred = wdl_model.predict(te_features)
res_df = pd.read_csv("data/test.csv")
res_df

,user_id,business_id,stars
0,39b627ea7d06c70fd36d5bfc6c23ddbb,d599f2a9f7e782f0a3fc16fd5dc22027,0.0
1,2fd27ce10e62ba2578d0c04799027a6e,69423d084bc0b22ea07aae598acc8c52,0.0
2,d7995d511b5493a4b6ec34af6290e080,9ef9cfddc1e6fab536d4517e204064be,0.0
3,36d8bb0193e3c785a2c47dba1abab521,4380c5fcdedd19169190eaae7de83986,0.0
4,b4a3aea52dea7ad3fc324a8c8d4356fc,88cd731bf93dd4706e824d0663018808,0.0
...,...,...,...
7505,42326317a1c3456c7ade772762f41d19,e4cde8be05f410c5f29dfe0cb76b21b5,0.0
7506,a557347314e8002c6ae452071e6b7e18,a074be3d6a7f48fd17d37aa7acfe10f1,0.0
7507,59c52c7a858503d1bb970923e08d1b0c,65399b606015d6148f10f927dd4ae0ff,0.0
7508,f0fa7ac5cd64443f3cfe4b2042e4a830,94bb8df33b17a8e7678ec8d646950148,0.0


In [30]:
res_df['stars'] = y_pred[:, 0]
NAME = 'test'
res_df.to_csv("{}.csv".format(NAME), index=False)
print("Writing test predictions to file done.")

Writing test predictions to file done.


In [31]:
res_df

,user_id,business_id,stars
0,39b627ea7d06c70fd36d5bfc6c23ddbb,d599f2a9f7e782f0a3fc16fd5dc22027,4.178943
1,2fd27ce10e62ba2578d0c04799027a6e,69423d084bc0b22ea07aae598acc8c52,3.682924
2,d7995d511b5493a4b6ec34af6290e080,9ef9cfddc1e6fab536d4517e204064be,4.011315
3,36d8bb0193e3c785a2c47dba1abab521,4380c5fcdedd19169190eaae7de83986,4.650148
4,b4a3aea52dea7ad3fc324a8c8d4356fc,88cd731bf93dd4706e824d0663018808,3.672891
...,...,...,...
7505,42326317a1c3456c7ade772762f41d19,e4cde8be05f410c5f29dfe0cb76b21b5,3.831137
7506,a557347314e8002c6ae452071e6b7e18,a074be3d6a7f48fd17d37aa7acfe10f1,3.662682
7507,59c52c7a858503d1bb970923e08d1b0c,65399b606015d6148f10f927dd4ae0ff,4.082367
7508,f0fa7ac5cd64443f3cfe4b2042e4a830,94bb8df33b17a8e7678ec8d646950148,4.221411
